INTERACTIVE BUDGET REVIEWER and PREDICTER
==========

**Here where we declare the needed libraries and global variable** 

In [8]:
import datetime as dt
import glob
import math
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as sp

import plotly.graph_objs as go
from ipywidgets import interactive, HBox, VBox, widgets, Layout, fixed
from IPython.display import display

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures


# Ottieni i nomi dei file CSV nella tua directory
path = "C:\\Users\\Davidde\\Downloads\\Telegram Desktop\\Budget semplice"

**FUNCTION DECLARATION**
======================

1. **avg_values_file** is a function that based on the regexp name returns all the file with the expected regexp inside the defined path
2. **collect_data_from_list_csv** is a function that returns a numpy array with extracted information from path, file_name and wanted_regexp 

In [34]:
def collect_file(path, name):
    filenames = glob.glob(path + f"/*202*-{name}.csv", recursive=True)
    # Ordina i file dal più recente al più vecchio
    filenames.sort(key=os.path.getmtime)
    # Inverti l'ordine della lista per avere i file dal più vecchio al più
    # nuovo
    filenames = filenames[::-1]
    number_sample = len(filenames)
    return filenames, number_sample

def collect_numb_sample(path,file_name):
    filenames, number_sample = collect_file(path, file_name)
    return number_sample

def collect_data_from_list_csv(path, file_name, wanted_regexp, scaling_factor):
    filenames, number_sample = collect_file(path, file_name)
    list_collected = []

    # Leggi ogni file e aggiungi il valore alla lista
    for filename in filenames:
        df = pd.read_csv(
            filename, sep=";", header=None, names=["Descrizione", "Valore"]
        )
        # Filtra il dataframe per la descrizione desiderata
        filtered_df = df[df["Descrizione"] == f"{wanted_regexp}"]["Valore"]
        if not filtered_df.empty:  # Verifica se il dataframe filtrato non è vuoto
            collected_data = filtered_df.values[0]  # Prendi il primo valore
            # Processa il dato raccolto
            collected_data = (
                collected_data.replace("€", "")
                .replace(".", "")
                .replace(",", ".")
                .strip()
            )
            try:
                if collected_data.startswith("-"):
                    collected_data = float(collected_data[1:]) * -1
                else:
                    collected_data = float(collected_data)
                list_collected = np.append(
                    list_collected, collected_data * scaling_factor
                )
            except ValueError:
                print(f"Impossibile convertire {collected_data} in float.")
        else:
            print(f"Nessun valore trovato per {wanted_regexp} in {filename}.")

    return np.array(list_collected)

def collect_data_from_csv(path, file_name, wanted_regexp, scaling_factor=1):
    data = collect_data_from_list_csv(
        path,
        file_name=file_name,
        wanted_regexp=wanted_regexp,
        scaling_factor=scaling_factor
    )
    return data



**Some basics mathematical functions**

1. **month_year** :
    - Restituisce mese e anno corrente 
2. **simple_mean** :
    - media consecutiva
3. **dynamic_avg** :
    - E' una moving average usata per calcolare la media sui valori collezionati

In [30]:
def month_year():
    now = dt.datetime.now()
    return now.month, now.year

# Funzione per formattare i valori percentuali
def format_percentage(value):
    return "{:.2f}%".format(value)

def simple_mean(previous_avg, new_value, n):
    # You need to define this function for computing the new average.
    # Assuming `previous_avg` is the average of the first `n-1` items,
    # `new_value` is the nth item, and `n` is the total count,
    # the new average will be computed as follows:
    return ((previous_avg * (n - 1)) + new_value) / n


def dynamic_avg(values):
    # This function calculates the dynamic average of a list of values.
    risparmio_netto_avg_values = []
    avg_intermediate = []

    for i, value in enumerate(values):
        if i == 0:
            # The average of the first element is the element itself.
            new_avg = value
        else:
            # Compute the new average based on the previous one.
            new_avg = simple_mean(avg_intermediate[i - 1], value, i + 1)

        # Append the new average to the intermediate list.
        avg_intermediate.append(new_avg)

        # Append the new average to the final list of averages.
        risparmio_netto_avg_values.append(new_avg)

    # Convert the list of averages to a numpy array.
    return np.array(risparmio_netto_avg_values)


def sum(values):
    sum_value = 0
    sum_value_list = []
    for i, value in enumerate(values):
        sum_value += value
        sum_value_list.append(sum_value)
    return np.array(sum_value_list), sum_value

# Definire una funzione generica per calcolare la somma e la media 
def calculate_sum_and_average(variable_name, data, operation_funcs):
    sum_value = operation_funcs['sum'](data) if 'sum' in operation_funcs else None
    avg_value = operation_funcs['avg'](data) if 'avg' in operation_funcs else None
    return sum_value, avg_value

# Time Informations
month, year = month_year()

# Crea una lista di date dal settembre 2021 fino al mese e all'anno correnti
date_list = pd.date_range(start="2021-09", end=f"{year}-{month}", freq="MS")

1. **reddito_annuo** :
    - Colleziona tutti i file che hanno **Reddito** come file name e **Stipendio** come regexp interna al file
    - Controlla che il numero di sample sia un multiplo di anni per poter prelevare il corretto numero di elementi da **reddito_collect**  
2. **stipendio_annuo_totale** :
    - stipendio_diviso_per_anni, growth_rate, average_growth_rate, date_anni

In [41]:
def reddito_annuo(reference_year, reference_month, path, scaling_factor):
    reddito_collect = collect_data_from_list_csv(
        path,
        file_name="Reddito",
        wanted_regexp="Stipendio",
        scaling_factor=scaling_factor,
    )
    initial_date_record = pd.to_datetime("2021-09-01")
    start_date = pd.to_datetime(f"{reference_year}-{reference_month}")
    reddito_annuo_result = 0.0
    try:
        if initial_date_record <= start_date:
            delta = start_date - initial_date_record
            delta_in_months = delta.days / (
                30
                if reference_year % 4 == 0
                and (reference_year % 100 != 0 or reference_year % 400 == 0)
                else 31
            )
            delta_in_months_rounded = (
                math.floor(delta_in_months + 0.5)
                if delta_in_months % 1 >= 0.5
                else math.ceil(delta_in_months - 0.5)
                if delta_in_months % 1 < 0.5
                else delta_in_months
            )
    except ValueError:
        print(f"Impossibile accedere a {reddito_collect} in float.")
    if delta_in_months_rounded <= 12:
        for i in range(int(delta_in_months_rounded)):
            reddito_annuo_result += reddito_collect[i]
    else:
        for i in range(int(delta_in_months_rounded - 12), int(delta_in_months_rounded)):
            reddito_annuo_result += reddito_collect[i]
    return reddito_annuo_result


def numero_anni(number_samples):
    numero_anni = number_samples // 12
    return numero_anni


def stipendio_annuo_totale(number_samples, scaling_factor):
    numero_anni_osservati = numero_anni(number_samples=number_samples)
    stipendio_diviso_per_anni = []
    growth_rate = []
    average_growth_rate = []
    start_ref_year = 2022
    ref_month = 9
    date_list = pd.date_range(
        start=f"{start_ref_year}-{ref_month}",
        end=f"{
            start_ref_year - 1 + numero_anni_osservati}-{ref_month}",
        freq="MS",
    )
    date_anni = {date_list[0], date_list[-1]}
    date_anni = pd.DatetimeIndex(date_anni)  # type: ignore
    for i in range(numero_anni_osservati):
        redd_obs_year = reddito_annuo(
            reference_year=start_ref_year + i,
            reference_month=ref_month,
            path=path,
            scaling_factor=scaling_factor,
        )
        stipendio_diviso_per_anni = np.append(stipendio_diviso_per_anni, redd_obs_year)
    # Calcola la differenza tra ogni valore e il precedente
    diff = np.diff(stipendio_diviso_per_anni)
    # Rimuovere l'elemento di posizione 0
    diff = np.squeeze(diff)
    # Calcola il tasso di crescita
    growth_rate = diff / stipendio_diviso_per_anni[:-1]
    # Calcola il tasso di crescita medio
    average_growth_rate = np.mean(growth_rate)
    return stipendio_diviso_per_anni, growth_rate, average_growth_rate, date_anni

PLOTTING FUNCTION DEFINITION
===========================

In [32]:
def create_plot(x, y, name_trace, name_graph, overlap, n_traces):
    fig = go.Figure()
    # Aggiungi i valori al grafico
    if overlap:
        for i, y_list in enumerate(y):
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=y_list,
                    mode="lines+markers",
                    name=f"{
                        name_trace[i]}",
                )
            )
    else:
        fig.add_trace(go.Scatter(x=x, y=y, mode="lines+markers", name=f"{name_trace}"))
    # Imposta le etichette degli assi e il titolo
    fig.update_layout(
        title=f"{name_graph}",
        xaxis_title="",
        yaxis_title="",
        legend_title="Legenda",
        hovermode="x",
    )
    fig.show()
    return fig


def create_subplot(x, y, y1, name_graph, name_trace, name_trace1, overlap, n_graphs):
    # Creazione di un oggetto subplots
    subplots = sp.make_subplots(rows=1, cols=n_graphs)

    # Aggiungi i valori al grafico
    if overlap:
        for i, y_list in enumerate(y):
            subplots.add_trace(
                go.Scatter(
                    x=x,
                    y=y_list,
                    mode="lines+markers",
                    name=f"{
                        name_trace[i]}",
                ),
                row=1,
                col=1,
            )
        for k, y1_list in enumerate(y1):
            subplots.add_trace(
                go.Scatter(
                    x=x,
                    y=y1_list,
                    mode="lines+markers",
                    name=f"{
                        name_trace1[k]}",
                ),
                row=1,
                col=2,
            )
    else:
        subplots.add_trace(
            go.Scatter(x=x, y=y, mode="lines+markers", name=f"{name_trace}"),
            row=1,
            col=1,
        )

    # Imposta le etichette degli assi e il titolo
    subplots.update_layout(
        title=f"{name_graph}",
        xaxis_title="",
        yaxis_title="",
        legend_title="Legenda",
        hovermode="x",
    )

    # Mostra il grafico
    subplots.show()

    return subplots

# Funzione per creare e aggiornare il grafico
def create_interactive_plot(data,choice):
    plot_data = data[choice]
    fig = go.Figure()
    for i, dataset in enumerate(plot_data['data']):
        fig.add_trace(go.Scatter(x=date_list, y=dataset, mode='lines+markers', name=plot_data['names'][i]))
    fig.update_layout(title=f"{choice} nel Tempo", xaxis=dict(title='Date'), yaxis=dict(title=choice))
    fig.show()

# Widget interattivo per la selezione del tipo di dati da plottare
def create_type_selector(data):
    type_selector = widgets.Dropdown(
        options=list(data.keys()),
        value=list(data.keys())[0],
        description='Tipo:',
        disabled=False,
    )
    return type_selector

# Visualizzazione della tabella
def show_table(df):
    # Crea una lista di liste, una per ogni colonna del DataFrame
    cell_values = [df[col].tolist() for col in df.columns]
    
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(df.columns),
                    fill_color='aquamarine',
                    align='left'),
        cells=dict(values=cell_values,
                   fill_color='lightgreen',
                   align='left'))
    ])
    fig.show()


**ML , LSTM, Montecarlo Function Definition for Prediction**
============================================================

In [13]:
from sklearn.ensemble import GradientBoostingRegressor
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler


def find_best_model(X, y):
    # Definire il range dei parametri da testare
    param_grid = {
        "polynomialfeatures__degree": list(
            np.arange(0, 3, 1)
        ),  # Gradi polinomiali da testare
        "lasso__alpha": list(
            np.arange(0.001, 2.501, 0.05)
        ),  # Valori di alpha da testare
        # Aggiungi un intervallo più ampio se necessario
    }

    # Creare un pipeline con PolynomialFeatures e Lasso Regression
    pipeline = Pipeline(
        [
            ("polynomialfeatures", PolynomialFeatures()),
            ("lasso", Lasso(max_iter=1000000, tol=0.0001)),
        ]
    )

    # Utilizzare GridSearchCV per trovare il miglior modello e il grado polinomiale
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=3,
        scoring="neg_mean_absolute_error",
        n_jobs=-1,
    )
    grid_search.fit(X, y)

    # Il miglior modello trovato dalla ricerca su griglia
    best_model_lasso = grid_search.best_estimator_
    best_score_lasso = -grid_search.best_score_
    best_params_lasso = grid_search.best_params_
    best_degree_lasso = best_params_lasso["polynomialfeatures__degree"]
    best_alpha_lasso = best_params_lasso["lasso__alpha"]

    print(
        f"Best polynomial degree: {best_degree_lasso}, Best alpha_lasso: {best_alpha_lasso}"
    )
    print(f"Best MAE score_lasso from GridSearchCV: {best_score_lasso}")

    pipeline = Pipeline([("regressor", HuberRegressor())])

    # Definire il range dei parametri da testare (potrebbe essere esteso a seconda delle esigenze)
    param_grid = {"regressor__alpha": list(np.arange(0.001, 2.501, 0.05))}

    # Utilizzare GridSearchCV per trovare il miglior modello
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=3,
        scoring="neg_mean_absolute_error",
        n_jobs=-1,
    )
    grid_search.fit(X, y)

    # Il miglior modello trovato dalla ricerca su griglia
    best_model_huber = grid_search.best_estimator_
    best_score_huber = -grid_search.best_score_
    best_params_huber = grid_search.best_params_

    print(f"Best hyperparameters_huber :{best_params_huber}")
    print(f"Best MAE score_huber from GridSearchCV: {best_score_huber}")

    # Definire il range dei parametri da testare per Gradient Boosting
    param_grid_gb = {
        "n_estimators": [100, 200, 300],  # Numero di boosting stages da testare
        "learning_rate": [0.01, 0.1, 0.2],  # Velocità di apprendimento da testare
        "max_depth": [3, 4, 5],  # Profondità massima degli alberi da testare
        "min_samples_split": [
            2,
            3,
            4,
        ],  # Numero minimo di campioni richiesti per dividere un nodo
        # Aggiungi altri parametri di GradientBoostingRegressor qui se necessario
    }

    # Inizializzare Gradient Boosting Regressor
    gb_model = GradientBoostingRegressor()

    # Utilizzare GridSearchCV per trovare il miglior modello di Gradient Boosting
    grid_search_gb = GridSearchCV(
        gb_model, param_grid_gb, cv=3, scoring="neg_mean_absolute_error", n_jobs=-1
    )
    grid_search_gb.fit(X, y)

    # Il miglior modello trovato dalla ricerca su griglia per Gradient Boosting
    best_model_gb = grid_search_gb.best_estimator_
    best_score_gb = -grid_search_gb.best_score_
    best_params_gb = grid_search_gb.best_params_

    print(f"Best parameters for Gradient Boosting: {best_params_gb}")
    print(f"Best MAE score from GridSearchCV for Gradient Boosting: {best_score_gb}")

    # Confronto dei punteggi per determinare il modello migliore
    best_score, best_model = min(
        (best_score_lasso, best_model_lasso),
        (best_score_huber, best_model_huber),
        (best_score_gb, best_model_gb),
        key=lambda x: x[0],
    )

    print(f"Best Model decided: {type(best_model).__name__}")

    return best_model


def project_future_values(
    data_collected, data_for_projection, months_to_project, inflation_rate
):
    # Create a time index for the existing data
    month, year = month_year()
    date_index = pd.date_range(start="2021-09", periods=len(data_collected), freq="MS")
    df_project = pd.DataFrame(
        data_for_projection, index=date_index, columns=["Data_for_Projection"]
    )

    # Prepare data for non-linear regression
    X = np.arange(len(df_project)).reshape(-1, 1)
    y = df_project["Data_for_Projection"].values

    best_model = find_best_model(X, y)

    best_model.fit(X, y)

    # Create the time index for future months
    future_index = pd.date_range(
        start=df_project.index[-1] + pd.offsets.MonthBegin(1),
        periods=months_to_project,
        freq="MS",
    )

    # Prepare data for prediction
    X_future = np.arange(len(df_project), len(df_project) + months_to_project).reshape(
        -1, 1
    )

    # Predict new values
    future_values = best_model.predict(X_future)

    # Calculate the monthly inflation from the annual rate
    monthly_inflation_rate = (1 + inflation_rate) ** (1 / 12) - 1

    # Adjust the predicted values for inflation
    inflation_adjustments = (1 + monthly_inflation_rate) ** np.arange(
        1, months_to_project + 1
    )
    adjusted_future_values = future_values * inflation_adjustments

    # Merge historical data with predictions
    future_df = pd.DataFrame(
        adjusted_future_values, index=future_index, columns=["ProjectedDataCollected"]
    )
    total_index = date_index.append(future_index)
    combined_data = np.concatenate([data_collected, adjusted_future_values])
    result_df = pd.DataFrame(
        combined_data, index=total_index, columns=["Data_Combined"]
    )

    return future_df, result_df, total_index


**ESTRAZIONE DATI DA CSV**
==================

In [47]:
# Lista delle configurazioni per la raccolta dei dati
data_configurations = [
    ("reddito_aggiuntivo_collect", "Reddito", "Reddito aggiuntivo"),
    ("risparmio_netto_collect", "Netto", "Reddito meno spese"),
    ("reddito_collect", "Reddito", "Reddito totale"),
    ("reddito_only_ifx", "Reddito", "Stipendio"),
    ("spese_collect", "Spese", "Spese totali"),
    ("investment_collect", "Spese", "Investimenti"),
    ("costo_casa_collect", "Spese", "Immobili (affitto, mutuo, tasse, assicurazione)"),
    ("spese_straordinarie_collect", "Spese", "Spese Straordinarie"),
    # Aggiungi altre configurazioni qui...
]

# Effettua la raccolta dei dati per ogni configurazione e assegna i risultati alle variabili globali
data_collected = {
    config[0]: collect_data_from_csv(path, config[1], config[2], scaling_factor=1)
    for config in data_configurations
}

number_months = collect_numb_sample(path,"Reddito")


**CALCOLO DATI DA CSV**
==================

In [68]:
sum_results = {}
avg_results = {}
perct_avg_results = {}
perct_results = {}

risparmio_invest_liquid_total_list = data_collected["investment_collect"] + data_collected["risparmio_netto_collect"]
spese_nette_values                 = data_collected["spese_collect"] - data_collected["investment_collect"]

sum_variables = {
    'reddito_total_list'     : data_collected["reddito_collect"],
    'investement_total_list' : data_collected["investment_collect"],
    'risparmio_total_list'   : data_collected["risparmio_netto_collect"],
    'spese_nette_total_list' : spese_nette_values,    
}


operations = {'avg': dynamic_avg, 'sum': sum}    # Definisci le operazioni utilizzate

for var_name, data in sum_variables.items():
    if var_name.endswith('_total_list'):  # Se la variabile è per la somma totale
        sum_results[f"{var_name}"], sum_results[f"{var_name}_total"] = operations['sum'](data)

avg_variables = {
    'risparmio_netto_avg_values': data_collected["risparmio_netto_collect"],
    'reddito_avg_values': data_collected["reddito_collect"],
    'reddito_aggiuntivo_avg_values': data_collected["reddito_aggiuntivo_collect"],
    'reddito_only_ifx_avg_values': data_collected["reddito_only_ifx"],
    'investment_collect_avg_values': data_collected["investment_collect"],
    'risparmio_invest_liquid_avg_values': risparmio_invest_liquid_total_list,
    'risparmio_total_avg_values': sum_results["risparmio_total_list"],
    'spese_total_avg_values': data_collected["spese_collect"],
    'spese_nette_avg_values': spese_nette_values,
    'spese_straordinarie_collect_avg_values': data_collected["spese_straordinarie_collect"],
    'costo_casa_avg_values': data_collected["costo_casa_collect"],
}
        
for var_name, data in avg_variables.items():
    if var_name.endswith('_avg_values'):  # Se la variabile è per la media
        avg_results[f"{var_name}"] = operations['avg'](data)
        

# Variabili specifiche non gestite dal ciclo
stipendio_diviso_per_anni, growth_rate, average_growth_rate, observed_years = stipendio_annuo_totale(number_months, scaling_factor=1)

############### 1. CALCOLI PERCENTUALI SU VALORI MEDI #######################
# Lista di configurazione per i nomi delle variabili percentuali e le loro formule
perct_avg_variables = {
    'costo_casa_perct_avg_values': ("costo_casa_avg_values", "reddito_avg_values"),
    'investement_total_perct_avg_values': ("investment_collect_avg_values", "reddito_avg_values"),
    'spese_nette_perct_avg_values': ("spese_nette_avg_values", "reddito_avg_values"),
    'risparmio_no_invest_perct_avg_values': ("risparmio_netto_avg_values", "reddito_avg_values"),
    'risparmio_global_perct_avg_values': ("risparmio_invest_liquid_avg_values", "reddito_avg_values"),
}

# Calcolo delle percentuali medie
for var_name, (numerator, denominator) in perct_avg_variables.items():
    # Calcola e salva il risultato nel dizionario avg_results
    perct_avg_results[f"{var_name}"] = operations['avg'](avg_results[numerator] / avg_results[denominator])*100

############# 2. CALCOLI PERCENTUALI SU VALORI PUNTUALI MEDIATI #########
# Lista di configurazione per i nomi delle variabili percentuali e le loro formule sui valori puntuali
perct_variables = {
    'costo_casa_perct_values': ("costo_casa_collect", "reddito_collect"),
    'investement_total_perct_values': ("investment_collect", "reddito_collect"),
    'spese_nette_perct_values': ("spese_nette_values", "reddito_collect"),
    'risparmio_no_invest_perct_values': ("risparmio_netto_collect", "reddito_collect"),
    'risparmio_global_perct_values': ("risparmio_invest_liquid_total_list", "reddito_collect"),
}

# Calcolo delle percentuali sui valori puntuali mediati
for var_name, (numerator_collect, denominator_collect) in perct_variables.items():
    if numerator_collect.endswith('_nette_values'):
        perct_results[f"{var_name}"] = operations['avg'](spese_nette_values / data_collected[denominator_collect]) * 100
    elif numerator_collect.endswith('_total_list'):
        perct_results[f"{var_name}"] = operations['avg'](risparmio_invest_liquid_total_list / data_collected[denominator_collect]) * 100
    else:
        perct_results[f"{var_name}"] = operations['avg'](data_collected[numerator_collect] / data_collected[denominator_collect]) * 100


Costo Casa %: [ 9.14032615 12.73459902 14.95244779 13.82236053 14.51863928 15.30180315
 14.11873688 13.65650596 14.31798362 15.13784088 16.31094453 15.66214691
 16.15092696 16.43687801 15.89859726 16.30413538 16.73252703 16.38523648
 16.52805948 16.77317797 16.79174525 16.46133451 16.42530724 16.50558165
 16.53881515 16.66552145 16.43708103 16.54929742 17.34703601 17.1169233
 17.40032509 18.17357158 18.50770814 18.40800377 18.74773244]
Investimenti %: [14.53593719 20.2890011  37.0314182  32.71183525 44.75649892 38.87956123
 33.6600864  29.82517605 27.34501781 27.67579512 26.35245515 25.05193961
 27.13843309 26.30169963 24.76158632 24.01709968 23.33999106 24.45832127
 30.15345343 29.2322567  28.13780153 27.06893211 26.13123082 25.36722537
 24.52085111 23.93419686 23.17939336 22.63746822 23.0412255  22.46425042
 21.93806763 21.46004571 21.64836418 23.38025257 22.72138334]
Spese Nette %: [70.22347493 80.06393632 82.92344002 71.49299154 74.20463214 94.10077112
 86.83323865 79.44128723 77.4

**GRAFICI CON DATI DA CSV**
==================

In [24]:

# Dizionario per i dati
data_summary = {
    "Spese Nette Mensili": format_percentage(spese_nette_perct_avg_values[-1]),
    "Risparmio (no Investimenti)": format_percentage(risparmio_no_invest_perct_avg_values[-1]),
    "Investimenti": format_percentage(investement_total_perct_avg_values[-1]),
    "Reddito Medio": f"€ {reddito_avg_values[-1]:.2f}",
    "Stipendio Medio": f"€ {reddito_only_ifx_avg_values[-1]:.2f}",
    "Spesa Netta Media": f"€ {spese_nette_avg_values[-1]:.2f}",
    "Risparmio Medio Mensile": f"€{risparmio_invest_liquid_avg_values[-1]}"
}

# Creazione di un DataFrame pandas
df_summary = pd.DataFrame(list(data_summary.items()), columns=['Categoria', 'Valore Percentuale'])

# Chiamata della funzione per visualizzare la tabella
show_table(df_summary)

# Dati per i grafici - sostituisci questi con i tuoi dati reali
data = {
    "Risparmio": {
        "data": [risparmio_netto_collect, risparmio_netto_avg_values],
        "names": ["Risparmio Netto", "Risparmio Netto Medio"]
    },
    "Reddito": {
        "data": [reddito_collect, reddito_avg_values],
        "names": ["Reddito Percepito", "Reddito Medio"]
    },
    "Spese Mensili": {
        "data": [spese_collect, spese_total_avg_values],
        "names": ["Spese Mensili", "Spese Medie Mensili"]
    },
    "Spese - Investimenti": {
        "data": [spese_nette_values, spese_nette_avg_values],
        "names": ["Spese - Investimenti Mensili", "Spese - Investimenti Medi"]
    },
    "Confronti Reddito, Investimenti, Risparmio": {
        "data": [
            investment_collect_avg_values,
            risparmio_total_avg_values,
            reddito_avg_values,
            investement_total_list,
            risparmio_total_list,
            reddito_total_list,
            spese_nette_total_list,
            risparmio_netto_collect,
            risparmio_invest_liquid_total_list,
        ],
        "names": [
            "Investimenti Mensili Medi",
            "Risparmio Accumulato Medio (Liquidità)",
            "Reddito Mensile Mensile",
            "Investimenti Totali",
            "Risparmio Accumulato (Liquidità)",
            "Reddito Totale",
            "Spese Totali senza Investimenti",
            "Risparmio Mensile senza Investimenti (Liquidità)",
            "Risparmio Liquidità + Investimenti Mensile",
        ]
    },
    "Confronti Reddito, Investimenti, Risparmio Medio Percentuali": {
        "data": [
            investement_total_perct_avg_values,
            risparmio_no_invest_perct_avg_values,
            spese_nette_perct_avg_values,
            risparmio_global_perct_avg_values,
            costo_casa_perct_avg_values,
        ],
        "names": [
            "Investimenti/Reddito Percentuali AVG Mensili",
            "Risparmio (senza Investimenti)/Reddito Percentuali Accumulato AVG Mensili",
            "Spese Nette/Reddito Percentuali AVG Mensili",
            "Risparmio Globale Percentuali AVG Mensili",
            "Costo Casa AVG Percentuali Mensili",
        ]
    },
    "Confronti Reddito, Investimenti, Risparmio Mensili Percentuali": {
        "data": [
            investement_total_perct_values,
            risparmio_no_invest_perct_values,
            spese_nette_perct_values,
            risparmio_global_perct_values,
            costo_casa_perct_values,
        ],
        "names": [
            "Investimenti/Reddito Percentuali Mensili",
            "Risparmio(senza Investimenti)/Reddito Percentuali Accumulato Mensili",
            "Spese Nette/Reddito Percentuali Mensili",
            "Risparmio Globale Percentuali Mensili",
            "Costo Casa Percentuali Mensili",
        ]
    },
    # Aggiungi altri dataset qui...
}


# Collegamento dei widget con la funzione di creazione dei grafici
type_selector = create_type_selector(data)
interactive_plot = interactive(create_interactive_plot, data=fixed(data), choice=type_selector)

# Display dei widget e del grafico interattivo
display(interactive_plot)


interactive(children=(Dropdown(description='Tipo:', options=('Risparmio', 'Reddito', 'Spese Mensili', 'Spese -…

**CALCOLO DATI DA PREDICITON METHODs**
==================
**I valori predetti non necessitano di ulteriore media, visto che tale operazione e' presente nel modello di ML implementato**

In [18]:
# Simulazione
inflation_rate = 0.02  # % di inflazione annuale
inflation_rate_salary = 0.015  # scala mobile stipendio
ratio_stipendio_reddito = dynamic_avg(reddito_only_ifx_avg_values / reddito_avg_values)
inflation_rate_avg_reddito = ratio_stipendio_reddito[-1] * inflation_rate_salary
new_house = 1  # Switch Cambio Casa
if new_house == 0:
    ratio_new_old_apartment = 1
else:
    ratio_new_old_apartment = 1291.54 / 791.84
inflation_rate_casa = 0.02
months_to_project = 12  # Simulazione su N mesi
# Scaling Factor Spese Straordinarie --> 0.2 = -20% , 0.7 = -70% ...
scaling_factor_spese_straordinarie = 0.4
################# VALORI PREDETTI ##############################
####### regressor_alpha close to 0 ---> Linear Regression ######
costo_casa_predicted_values, costo_casa_hystory_values, date_index_project = (
    project_future_values(
        costo_casa_collect,
        costo_casa_avg_values,
        months_to_project,
        inflation_rate_casa,
    )
)

(
    reddito_aggiuntivo_predicted_values,
    reddito_aggiuntivo_hystory_values,
    date_index_project,
) = project_future_values(
    reddito_aggiuntivo_collect, reddito_aggiuntivo_avg_values, months_to_project, 0.0
)
stipendio_predicted_values, stipendio_hystory_values, date_index_project = (
    project_future_values(
        reddito_only_ifx,
        reddito_only_ifx_avg_values,
        months_to_project,
        inflation_rate_salary,
    )
)
investment_predicted_values, investment_hystory_values, date_index_project = (
    project_future_values(
        investment_collect, investment_collect_avg_values, months_to_project, 0.00001
    )
)
spese_nette_da_predire = spese_nette_values - (
    spese_straordinarie_collect * scaling_factor_spese_straordinarie
)
spese_nette_da_predire_avg_values = dynamic_avg(spese_nette_da_predire)
spese_nette_predicted_values, spese_nette_hystory_values, date_index_project = (
    project_future_values(
        spese_nette_da_predire,
        spese_nette_da_predire_avg_values,
        months_to_project,
        inflation_rate,
    )
)
risparmio_netto_predicted_values, risparmio_netto_hystory_values, date_index_project = (
    project_future_values(
        risparmio_netto_collect, risparmio_netto_avg_values, months_to_project, 0.0
    )
)
################ CALCOLI PERCENTUALI PREDETTI ######################
reddito_predicted_collect = (
    reddito_aggiuntivo_hystory_values["Data_Combined"].values
    + stipendio_hystory_values["Data_Combined"].values
)  # type: ignore
reddito_predicted_total_values_list, reddito_predicted_total_value = sum(
    reddito_predicted_collect
)
reddito_predicted_avg_values = dynamic_avg(reddito_predicted_collect)
stipendio_predicted_collect = stipendio_hystory_values["Data_Combined"].values
stipendio_predicted_avg_values = dynamic_avg(stipendio_predicted_collect)
stipendio_predicted_total_values_list, stipendio_predicted_total_value = sum(
    stipendio_predicted_collect
)
costo_casa_hystory_values[date_list.size :] *= ratio_new_old_apartment
costo_casa_predicted_collect = costo_casa_hystory_values["Data_Combined"].values
costo_casa_predicted_avg_values = dynamic_avg(costo_casa_predicted_collect)
costo_casa_predicted_total_values_list, costo_casa_predicted_total_value = sum(
    costo_casa_predicted_collect
)
investment_predicted_collect = investment_hystory_values["Data_Combined"].values
investment_predicted_avg_values = dynamic_avg(investment_predicted_collect)
investment_predicted_total_values_list, investment_predicted_total_value = sum(
    investment_predicted_collect
)
spese_nette_predicted_collect = spese_nette_hystory_values["Data_Combined"].values
spese_nette_predicted_avg_values = dynamic_avg(spese_nette_predicted_collect)
(spese_nette_predicted_total_values_list, spese_nette_predicted_total_value) = sum(
    spese_nette_predicted_collect
)
risparmio_netto_predicted_collect = risparmio_netto_hystory_values[
    "Data_Combined"
].values
risparmio_netto_predicted_total_values_list, risparmio_netto_predicted_total_value = (
    sum(risparmio_netto_predicted_collect)
)
risparmio_netto_predicted_avg_values = dynamic_avg(risparmio_netto_predicted_collect)
############# CALCOLI PERCENTUALI SU VALORI PUNTUALI MEDIATI #########
# costo_casa_predicted_perct_values = (
#     costo_casa_predicted_collect / reddito_predicted_collect
# ) * 100
# investment_predicted_perct_values = (
#     investment_predicted_collect / reddito_predicted_collect
# ) * 100
# spese_nette_predicted_perct_values = (
#     spese_nette_predicted_collect / reddito_predicted_collect
# ) * 100
# risparmio_global_predicted_perct_values = (
#     risparmio_netto_predicted_collect / reddito_predicted_collect
# ) * 100
# risparmio_predicted_no_invest_perct_values = (
#     risparmio_global_predicted_perct_values - investment_predicted_perct_values
# )
costo_casa_predicted_perct_values = (
    costo_casa_predicted_avg_values / reddito_predicted_avg_values
) * 100
investment_predicted_perct_values = (
    investment_predicted_avg_values / reddito_predicted_avg_values
) * 100
spese_nette_predicted_perct_values = (
    spese_nette_predicted_avg_values / reddito_predicted_avg_values
) * 100
risparmio_global_predicted_perct_values = (
    risparmio_netto_predicted_avg_values / reddito_predicted_avg_values
) * 100
risparmio_predicted_no_invest_perct_values = (
    risparmio_global_predicted_perct_values - investment_predicted_perct_values
)

Best polynomial degree: 1, Best alpha_lasso: 2.451
Best MAE score_lasso from GridSearchCV: 40.5902698587916
Best hyperparameters_huber :{'regressor__alpha': 2.451}
Best MAE score_huber from GridSearchCV: 44.386067372496626
Best parameters for Gradient Boosting: {'learning_rate': 0.2, 'max_depth': 4, 'min_samples_split': 3, 'n_estimators': 300}
Best MAE score from GridSearchCV for Gradient Boosting: 71.2074032838482
Best Model decided: Pipeline
Best polynomial degree: 0, Best alpha_lasso: 0.001
Best MAE score_lasso from GridSearchCV: 441.21748839516516
Best hyperparameters_huber :{'regressor__alpha': 2.451}
Best MAE score_huber from GridSearchCV: 442.97317815773437
Best parameters for Gradient Boosting: {'learning_rate': 0.01, 'max_depth': 3, 'min_samples_split': 4, 'n_estimators': 200}
Best MAE score from GridSearchCV for Gradient Boosting: 302.60938713420586
Best Model decided: GradientBoostingRegressor
Best polynomial degree: 0, Best alpha_lasso: 0.001
Best MAE score_lasso from GridS

**GRAFICO DATI DA PREDICITON METHODs**
==================

In [19]:
# CONFRONTI PROIEZIONI FUTURE REDDITO, INVESTIMENTI, RISPARMIO PERCENTUALI
# ###   IMPLEMENTARE MODO PER POTER FARE N SUBPLOT IN MANIERA SEMPLICE FIXME!!!!!
y_list_fig7 = [
    reddito_predicted_total_values_list,
    stipendio_predicted_total_values_list,
    costo_casa_predicted_total_values_list,
    spese_nette_predicted_total_values_list,
    risparmio_netto_predicted_total_values_list,
    investment_predicted_total_values_list,
]
name_trace_list_fig7 = [
    "Reddito Totale Accumulato Mensile",
    "Stipendio    Accumulato  Mensile",
    "Costo Casa   Accumulato  Mensile",
    "Spese Nette  Accumulato Mensili",
    "Risparmio (Liquidita') Accumulato  Mensile",
    "Investimento Accumulato Mensile",
]
y1_list_fig7 = [
    investment_predicted_perct_values,
    risparmio_predicted_no_invest_perct_values,
    spese_nette_predicted_perct_values,
    risparmio_global_predicted_perct_values,
    costo_casa_predicted_perct_values,
]
name_trace1_list_fig7 = [
    "Investimenti/Reddito Percentuali Mensili",
    "Risparmio(senza Investimenti)/Reddito Percentuali Accumulato Mensili",
    "Spese Nette/Reddito Percentuali Mensili",
    "Risparmio Globale Percentuali Mensili",
    "Costo Casa Percentuali Mensili",
]

n_traces_fig7 = 6
n_graphs_fig7 = 2
spese_nette_predicted_perct_avg_values_round = "{:.2f}".format(
    spese_nette_predicted_perct_values[-1]
)
risparmio_mensile_predicted_perct_avg_values_round = "{:.2f}".format(
    risparmio_predicted_no_invest_perct_values[-1]
)
investement_total_predicted_perct_avg_values_round = "{:.2f}".format(
    investment_predicted_perct_values[-1]
)
reddito_medio_predicted_avg_values_round = "{:.2f}".format(
    reddito_predicted_collect[-1]
)
stipendio_medio_predicted_avg_values_round = "{:.2f}".format(
    stipendio_predicted_collect[-1]
)
name_graph_fig7 = f"PROIEZIONE DATI SU {months_to_project} MESI  ---- S.Nette {spese_nette_predicted_perct_avg_values_round}% vs Inv. {investement_total_predicted_perct_avg_values_round}% vs Risp.(Liquidita') {
    risparmio_mensile_predicted_perct_avg_values_round}%  - R.M. € {reddito_medio_predicted_avg_values_round}, S.M. € {stipendio_medio_predicted_avg_values_round}, Spesa Netta Media € {spese_nette_predicted_avg_values[-1]}"
fig7_sub = create_subplot(
    x=date_index_project,
    y=y_list_fig7,
    y1=y1_list_fig7,
    name_graph=name_graph_fig7,
    name_trace=name_trace_list_fig7,
    name_trace1=name_trace1_list_fig7,
    overlap=1,
    n_graphs=n_graphs_fig7,
)